In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
from torchvision import models

In [2]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((600, 600)),  # Resize images to match original resolution
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Define dataset path
data_dir = r"E:\GAN Output\mri-based-brain-tumor-images\Brain MRI Images"

# Load dataset
train_dataset = torchvision.datasets.ImageFolder(root=r"E:\GAN Output\mri-based-brain-tumor-images\Brain MRI Images\Brain MRI Images\Train", transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=r"E:\GAN Output\mri-based-brain-tumor-images\Brain MRI Images\Brain MRI Images\Validation", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Print class names (should be ['Normal', 'Tumor'])
print(train_dataset.classes)


['Normal', 'Tumor']


In [3]:
# Load Pretrained MobileNetV2
model = models.mobilenet_v2(pretrained=True)

# Modify the classifier for binary classification
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)  # 2 output classes (Tumor, Normal)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


C:\Users\pulki\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\pulki\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [5]:
num_epochs = 5  # Increase if needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training Finished!")


Epoch [1/5], Loss: 0.5256
Epoch [2/5], Loss: 0.2619
Epoch [3/5], Loss: 0.1357
Epoch [4/5], Loss: 0.0540
Epoch [5/5], Loss: 0.0844
Training Finished!


In [6]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 92.50%


In [ ]:
torch.save(model.state_dict(), "tumor_classification_mobilenetv2.pth")
print("Model Saved!")


Model Saved!
